<a href="https://colab.research.google.com/github/nagpalm7/Text-Summarization/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# pip install attention.py

In [0]:
from attention import AttentionLayer

In [0]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
import nltk
from nltk import word_tokenize
nltk.download('all')
from nltk.corpus import stopwords
warnings.filterwarnings("ignore")

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# !unzip gdrive/My\ Drive/amazon-fine-food-reviews.zip

Archive:  gdrive/My Drive/amazon-fine-food-reviews.zip
  inflating: Reviews.csv             
  inflating: database.sqlite         
  inflating: hashes.txt              


In [0]:
#Data Pre-processing
def text_clean(text,n):
  processed_string=text.lower()
  processed_string=re.sub(r'[^\w\s]','',processed_string)
  
 #word_tokens=word_tokenize(processed_string)
  stop_words = set(stopwords.words('english'))
  new=processed_string.split()
  tokens=[]
  for w in new:
    if w not in stop_words:
      tokens.append(w)
  long_words=[]
  for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
  return (" ".join(long_words)).strip()
      
      
  
  
  
  
  

In [0]:
#Data importing and passing it for text cleaning
data=pd.read_csv("Reviews.csv",nrows=20000)
cleaned_text=[]
cleaned_summary=[]
# for lines in data['Text']:
#   clean_text.append(text_clean(lines,0))  
for i in range(0,20000):
  cleaned_text.append(text_clean(data['Text'][i],0))
  cleaned_summary.append(text_clean(data['Summary'][i],0))

In [0]:
#Tokenization of sentences
##Sentences tokenized into words and stored in a list
#from nltk.tokenize import word_tokenize
#words=[]
#for i in range(0,5000):
 # words.append(word_tokenize(string[i]))
  
#print(len(words))
#print(len(words[0]))

In [8]:
print(cleaned_text[0])
print(data['Text'][0])



bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better
I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


In [9]:
print(cleaned_summary[0])
print(data['Summary'][0])


good quality dog food
Good Quality Dog Food


In [0]:
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary


In [0]:
max_length_text=30
max_summary_len=8



In [12]:
len(cleaned_text)
len(cleaned_summary)

20000

In [0]:
short_text=[]
short_summary=[]
for i in range(0,len(cleaned_text)):
  if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_length_text):
    short_text.append(cleaned_text[i])
    short_summary.append(cleaned_summary[i])
    
df=pd.DataFrame({"text":short_text,"summary":short_summary})

In [0]:
df['summary'] = df['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(df['text']),np.array(df['summary']),test_size=0.1,random_state=0,shuffle=True)

In [0]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
x_tokenizer=Tokenizer()
x_tokenizer.fit_on_texts((X_train))

In [17]:
#Definings rare words
threshold=4
count=0
tot_count=0
frequency=0
tot_frequency=0
common_words=0
for key,value in x_tokenizer.word_counts.items():
  tot_count=tot_count+1
  tot_frequency=tot_frequency+value
  
  if(value<threshold):
    count=count+1
    frequency=frequency+value
    
common_words=tot_count-count
print(tot_frequency)    
  

176978


In [0]:
#Now we will prepare the tokenizer only for the topmost common words which are not rare

x_tokenizer=Tokenizer(common_words)

x_tokenizer.fit_on_texts((X_train))


x_tr_seq=x_tokenizer.texts_to_sequences(X_train)
x_val_seq=x_tokenizer.texts_to_sequences(X_test)

X_train  = pad_sequences(x_tr_seq,  maxlen=max_length_text, padding='post')
X_test   = pad_sequences(x_val_seq, maxlen=max_length_text, padding='post')


x_voc=len(x_tokenizer.word_index) + 1



In [19]:
y_tokenizer=Tokenizer()
y_tokenizer.fit_on_texts((y_train))
threshold=6
cnt=0
tot_cnt=0
freq=0
tot_freq=0
common_words=0
for key,value in y_tokenizer.word_counts.items():
  tot_cnt=tot_cnt+1
  tot_freq=tot_freq+value
  if(value<threshold):
    cnt=cnt+1
    freq=freq+value
    
common_words=tot_cnt-cnt
y_tokenizer=Tokenizer(common_words)
y_tokenizer.fit_on_texts((y_train))


y_tr_seq=x_tokenizer.texts_to_sequences(y_train)
y_val_seq=x_tokenizer.texts_to_sequences(y_test)

y_train  = pad_sequences(x_tr_seq,  maxlen=max_summary_len, padding='post')
y_test   = pad_sequences(x_val_seq, maxlen=max_summary_len, padding='post')


y_voc=len(y_tokenizer.word_index) + 1
y_voc
    

4061

In [0]:
y_tokenizer.word_counts['sostok'],len(y_train)
ind=[]
for i in range(len(y_train)):
    cnt=0
    for j in y_train[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_train=np.delete(y_train,ind, axis=0)
X_train=np.delete(X_train,ind, axis=0)


ind=[]
for i in range(len(y_test)):
    cnt=0
    for j in y_test[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_train=np.delete(y_train,ind, axis=0)
X_test=np.delete(X_test,ind, axis=0)

In [21]:
from keras import backend as K
K.clear_session()
latent_dim=300
embedding_dim=100



#Encoder
encoder_inputs=Input(shape=(max_length_text,))

enc_emb=Embedding(x_voc,embedding_dim,trainable=True)(encoder_inputs)

encoder_lstm1=LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)


#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()


W0719 08:43:33.458003 139776032425856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0719 08:43:33.459602 139776032425856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0719 08:43:33.511698 139776032425856 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0719 08:43:33.523495 139776032425856 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 100)      1426700     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 30, 300), (N 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [22]:
print(np.shape(X_train))
print(np.shape(y_train))
print(X_train)

print("__________________________________________________________________")

print(y_train)




(9681, 30)
(9681, 8)
[[1607   33   72 ...    0    0    0]
 [ 279   94   19 ...    0    0    0]
 [   1   80  322 ...    0    0    0]
 ...
 [  82  259  623 ...    0    0    0]
 [   5   87   46 ...    0    0    0]
 [ 207  147 1268 ...    0    0    0]]
__________________________________________________________________
[[  33   72   35 ...   54    7    2]
 [ 322  736  702 ...   42    2   87]
 [  15  316   59 ... 1685  112 3017]
 ...
 [1564    2   18 ...  206  710  426]
 [  43  148  163 ...  328 1127  491]
 [  26  952 1034 ...  218  147  207]]


In [0]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [0]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)


In [112]:
model.fit([X_train,y_train[:,:-1]], y_train.reshape(y_train.shape[0],y_train.shape[1], 1)[:,1:] ,epochs=60,callbacks=[es],batch_size=64, validation_data=([X_test,y_test[:,:-1]], y_test.reshape(y_test.shape[0],y_test.shape[1], 1)[:,1:]))


Train on 9681 samples, validate on 1076 samples
Epoch 1/60
9681/9681 [==============================] - 28s 3ms/sample - loss: 0.9412 - val_loss: 2.3220
Epoch 2/60
9681/9681 [==============================] - 28s 3ms/sample - loss: 0.8908 - val_loss: 2.3013
Epoch 3/60
9681/9681 [==============================] - 27s 3ms/sample - loss: 0.8520 - val_loss: 2.2927
Epoch 4/60
9681/9681 [==============================] - 27s 3ms/sample - loss: 0.8130 - val_loss: 2.2673
Epoch 5/60
9681/9681 [==============================] - 27s 3ms/sample - loss: 0.7838 - val_loss: 2.2440
Epoch 6/60
9681/9681 [==============================] - 27s 3ms/sample - loss: 0.7478 - val_loss: 2.2243
Epoch 7/60
9681/9681 [==============================] - 27s 3ms/sample - loss: 0.7219 - val_loss: 2.2378
Epoch 8/60
9681/9681 [==============================] - 28s 3ms/sample - loss: 0.6959 - val_loss: 2.2346
Epoch 00008: early stopping


In [0]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [0]:
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_length_text,latent_dim)) # change

dec_emb2= dec_emb_layer(decoder_inputs) 

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

decoder_outputs2 = decoder_dense(decoder_inf_concat) 

decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        
        # Sample a token
        
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        if(sampled_token_index==0):
          break;
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token
            
            
        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [0]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [121]:
for i in range(0,100):
    print("Review:",seq2text(X_train[i]))
    print("Original summary:",seq2summary(y_train[i]))
    
    print("Predicted summary:",decode_sequence(X_train[i].reshape(1,max_length_text)))
    
    print("\n")

Review: grandson loves makes cup everytime quick easy taste good 
Original summary: treats dont nice stinger full amazing love 
Predicted summary:  nice stinger stinger full amazing love


Review: enjoyed cookies much fact saw sale grocery store bought good stuff 
Original summary: could rodeo interesting found value kcup rice 
Predicted summary:  could rodeo found value kcup


Review: great treats fact thing use dog price seem way wrong far possibly local petsmart 
Original summary: favorite liver salt grain expensive trail cats capputo 
Predicted summary:  liver salt grain expensive trail cats capputo


Review: used product cake topper wedding cake made worked well happy got compliments 
Original summary: pocky delightful gluten teeth snack find cat werithers 
Predicted summary:  barely gluten teeth snack find fun million


Review: sadly disappointed brand assam tea properly brewed tea weak brewed another pot added tea hopes making drinkable second pot weak bitter 
Original summary: 